# Show difference in counts and Log2FoldChange over timepoints.

In [2]:
import scanpy as sc
import decoupler as dc
import numpy as np
import pandas as pd
import anndata as ad
from anndata import AnnData

In [3]:
# Read raw data and process it
adata = pd.read_csv('../data/MAFA_WT_gene_count.txt', index_col=20, sep='\t').T
adata

gene_name,CHGA,MT-CO1,GCG,INS,MT-ND4,EEF1A1,MT-ND5,TTR,MT-CO2,MT-CYB,...,RNA5-8SN4,AL354822.1,AC023491.2,RF00003,RF01972,RF01973,RF01974,AC233755.2,AC233755.1,AC213203.1
MAFAWT_001,769950,469036,398234,164748,186078,116719,151341,178909,133640,121061,...,0,0,0,0,0,0,0,0,0,0
MAFAWT_002,760265,204988,207026,146662,80616,98010,70795,109150,55131,49581,...,0,0,0,0,0,0,0,0,0,0
MAFAWT_003,672585,522418,378290,146501,243656,93180,195788,131732,163870,158054,...,0,0,0,0,0,0,0,0,0,0
MAFAWT_004,820162,262856,502891,171062,117809,132909,97752,150350,79653,75222,...,0,0,0,0,0,0,0,0,0,0
MAFAWT_005,878886,335731,73413,123677,122960,127821,102340,30019,96654,85530,...,0,0,0,0,0,0,0,0,0,0
MAFAWT_006,878745,350758,114376,147449,134333,125976,105483,31916,99623,94874,...,0,0,0,0,0,0,0,0,0,0
MAFAWT_007,1129979,359712,159369,65313,156749,104854,156351,25998,108889,113709,...,0,0,0,0,0,0,0,0,0,0
MAFAWT_008,1017468,285966,157747,109003,111200,119484,89777,32111,89609,81167,...,0,0,0,0,0,0,0,0,0,0
MAFAWT_009,800671,176313,194431,139982,64275,101077,52585,53031,49169,39752,...,0,0,0,0,0,0,0,0,0,0
MAFAWT_010,633710,177343,219577,113536,73264,76057,64194,57478,50504,46106,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Read in LFC + avg basemean from tables
# subset results df by list of genes from veronica
goi_t01 = pd.read_csv("../figures/direct/WT_T0_T1/goi_timepoints_01.csv")
goi_t01

,gene_name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,NEUROD1,9564.745000,-0.737122,0.094685,-7.888872,3.049299e-15,1.442365e-13
1,NKX6-1,9663.951000,0.886379,0.271446,3.872397,1.077703e-04,1.002271e-03
2,MAFA,8003.367000,5.225458,0.577766,9.133992,6.601880e-20,5.204652e-18
3,PDX1,5331.294000,0.846787,0.113554,7.723686,1.130128e-14,4.949705e-13
4,NR3C1,4998.454000,-0.785606,0.195044,-4.496700,6.901618e-06,8.460811e-05
5,STAT3,3754.968500,0.604386,0.065249,9.413438,4.801695e-21,4.218083e-19
6,GLIS3,2835.803500,0.911645,0.196943,5.058539,4.224803e-07,6.613838e-06
7,ASCL2,2591.480200,0.362044,0.193153,2.378306,1.739238e-02,7.065886e-02
8,MLXIP,2351.438000,0.984663,0.100186,10.031300,1.110443e-23,1.219346e-21
9,AHR,2046.126100,-0.331159,0.279640,-1.845780,6.492409e-02,1.936136e-01


In [ ]:
#plot
# X = T01, T02
# Y = (average LFC), avg basemean + jitter from baseMean from individual sample values?
